# Ensure that your code works on any system with Docker

In [53]:
%%capture
! docker build -t dbt-image .

In [54]:
! rm -rf ./tpch_warehouse/models/*/.ipynb_checkpoints # always run before dbt run, caused by notebooks, no need to do this if performed via terminal

zsh:1: no matches found: ./tpch_warehouse/models/*/.ipynb_checkpoints


In [55]:
! docker run -it --rm -v $(pwd)/tpch_warehouse:/dbt dbt-image dbt debug

11:40:14  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:14  Running with dbt=1.8.0
11:40:14  dbt version: 1.8.0
11:40:14  python version: 3.10.15
11:40:14  python path: /usr/local/bin/python3.10
11:40:14  os info: Linux-6.9.3-76060903-generic-x86_64-with-glibc2.36
11:40:14  Using profiles dir at /dbt
11:40:14  Using profiles.yml file at /dbt/profiles.yml
11:40:14  Using dbt_project.yml file at /dbt/dbt_project.yml
11:40:14  adapter type: duckdb
11:40:14  adapter version: 1.8.0
11:40:14  Configuration:
11:40:14    profiles.yml file [OK found and valid]
11:40:14    dbt_project.yml file [OK found and valid]
11:40:14  Required dependencies:
11:40:14   - git [OK found]

11:40:14  Connection:
11:40:14    database: dbt
11:40:14    schema: main
11:40:14    path: ./dbt.duckdb
11:40:14    config_options: None
11:40:14    extensions: None
11:40:14    settings: None
11:40:14    external_root: .
11

In [56]:
! docker run -it --rm -v $(pwd)/tpch_warehouse:/dbt dbt-image dbt run

11:40:17  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:17  Running with dbt=1.8.0
11:40:17  Registered adapter: duckdb=1.8.0
11:40:17  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.tpch_warehouse.example
11:40:17  Found 12 models, 14 data tests, 8 sources, 421 macros
11:40:17  
11:40:18  Concurrency: 1 threads (target='dev')
11:40:18  
11:40:18  1 of 12 START sql view model main.stg_customers ................................ [RUN]
11:40:18  1 of 12 OK created sql view model main.stg_customers ........................... [OK in 0.13s]
11:40:18  2 of 12 START sql view model main.stg_lineitems ................................ [RUN]
11:40:18  2 of 12 OK created sql view model main.stg_lineitems ........................... [OK in 0.13s]
11:40:18  3 of 12 START sql view model main.stg_n

In [57]:
! docker run -it --rm -v $(pwd)/tpch_warehouse:/dbt dbt-image dbt test

11:40:21  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:21  Running with dbt=1.8.0
11:40:21  Registered adapter: duckdb=1.8.0
11:40:21  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.tpch_warehouse.example
11:40:22  Found 12 models, 14 data tests, 8 sources, 421 macros
11:40:22  
11:40:22  Concurrency: 1 threads (target='dev')
11:40:22  
11:40:22  1 of 14 START test not_null_dim_customers_customer_key ......................... [RUN]
11:40:22  1 of 14 PASS not_null_dim_customers_customer_key ............................... [PASS in 0.07s]
11:40:22  2 of 14 START test not_null_fct_orders_orderkey ................................ [RUN]
11:40:22  2 of 14 PASS not_null_fct_orders_orderkey ...................................... [PASS in 0.06s]
11:40:22  3 of 14 START test not_null_stg_cus

In [58]:
! docker run -it --rm -v $(pwd)/tpch_warehouse:/dbt dbt-image dbt docs generate

11:40:24  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:24  Running with dbt=1.8.0
11:40:25  Registered adapter: duckdb=1.8.0
11:40:25  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.tpch_warehouse.example
11:40:25  Found 12 models, 14 data tests, 8 sources, 421 macros
11:40:25  
11:40:25  Concurrency: 1 threads (target='dev')
11:40:25  
11:40:25  Building catalog
11:40:26  Catalog written to /dbt/target/catalog.json


In [59]:
! docker run -it --rm -v $(pwd)/tpch_warehouse:/dbt -p 8080:8080 dbt-image dbt docs serve

11:40:28  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:28  Running with dbt=1.8.0
Serving docs at 8080
To access from your browser, navigate to: http://localhost:8080



Press Ctrl+C to exit.
172.17.0.1 - - [30/Oct/2024 11:40:31] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [30/Oct/2024 11:40:32] "GET /manifest.json?cb=1730288432069 HTTP/1.1" 200 -
172.17.0.1 - - [30/Oct/2024 11:40:32] "GET /catalog.json?cb=1730288432069 HTTP/1.1" 200 -
^C
11:40:37  Encountered an error:

11:40:37  Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/dbt/cli/requires.py", line 138, in wrapper
    result, success = func(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/dbt/cli/requires.py", line 101, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/dbt/cli/requires.py", line 218, in wrapper
    return func(*args, **kwargs)

In [60]:
! docker stop dbt-image

Error response from daemon: No such container: dbt-image


In [61]:
! docker rm dbt-image

Error response from daemon: No such container: dbt-image


In [62]:
! docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [63]:
%%capture
! docker compose up --build -d

In [64]:
! docker compose run dbt dbt run 

WARN[0000] /home/josephkevinmachado/code/de_101/5-Containerization/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
WARN[0000] Found orphan containers ([5-containerization-dbt-run-a13de9a3d99b 5-containerization-dbt-run-9a3aa89432ba 5-containerization-dbt-run-9c40d039bf29 5-containerization-dbt-run-e1e0188867f3 5-containerization-dbt-run-01903648e968 5-containerization-dbt-run-791eba2c0fd7 5-containerization-dbt-run-b05a1f41a1dd]) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up. 
11:40:40  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:41  Running with dbt=1.8.0
11:40:41  Registered adapter: duckdb=1.8.0
11:40:41  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resource

In [65]:
! docker compose run dbt dbt test

WARN[0000] /home/josephkevinmachado/code/de_101/5-Containerization/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
WARN[0000] Found orphan containers ([5-containerization-dbt-run-d86d22b7f313 5-containerization-dbt-run-a13de9a3d99b 5-containerization-dbt-run-9a3aa89432ba 5-containerization-dbt-run-9c40d039bf29 5-containerization-dbt-run-e1e0188867f3 5-containerization-dbt-run-01903648e968 5-containerization-dbt-run-791eba2c0fd7 5-containerization-dbt-run-b05a1f41a1dd]) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up. 
11:40:45  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:45  Running with dbt=1.8.0
11:40:45  Registered adapter: duckdb=1.8.0
11:40:46  [WARNING]: Configuration paths exist in your dbt_project.yml

In [66]:
! docker compose run dbt dbt docs generate

WARN[0000] /home/josephkevinmachado/code/de_101/5-Containerization/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
WARN[0000] Found orphan containers ([5-containerization-dbt-run-cac1c5135644 5-containerization-dbt-run-d86d22b7f313 5-containerization-dbt-run-a13de9a3d99b 5-containerization-dbt-run-9a3aa89432ba 5-containerization-dbt-run-9c40d039bf29 5-containerization-dbt-run-e1e0188867f3 5-containerization-dbt-run-01903648e968 5-containerization-dbt-run-791eba2c0fd7 5-containerization-dbt-run-b05a1f41a1dd]) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up. 
11:40:49  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:49  Running with dbt=1.8.0
11:40:49  Registered adapter: duckdb=1.8.0
11:40:50  [WARNING]: Configura

In [67]:
! docker compose run -p 8081:8080 dbt dbt docs serve

WARN[0000] /home/josephkevinmachado/code/de_101/5-Containerization/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
WARN[0000] Found orphan containers ([5-containerization-dbt-run-1bf2c4f60043 5-containerization-dbt-run-cac1c5135644 5-containerization-dbt-run-d86d22b7f313 5-containerization-dbt-run-a13de9a3d99b 5-containerization-dbt-run-9a3aa89432ba 5-containerization-dbt-run-9c40d039bf29 5-containerization-dbt-run-e1e0188867f3 5-containerization-dbt-run-01903648e968 5-containerization-dbt-run-791eba2c0fd7 5-containerization-dbt-run-b05a1f41a1dd]) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up. 
11:40:52  [WARNING]: Deprecated functionality

User config should be moved from the 'config' key in profiles.yml to the 'flags' key in dbt_project.yml.
11:40:52  Running with dbt=1.8.0
Serving docs at 8080
To access fr

In [68]:
%%capture
! docker compose down 

In [69]:
! docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


## Define the OS you want to run your code on with an Image

## Containers are where your OS (& code) runs, they are created from Image

### Containers can be always running or only exist for the duration of your code

## Containers can interact with your local OS

### Ensure ports are open for your code to interact with other systems

### Ensure code/data is synced between your local OS and your container with `volume mounts`

## Orchestrate multiple containers with `docker compose`

## Let's run our dbt pipeline with `docker exec`